In speech processing we may use really huge networks cause majority of the processing bottleneck is on CPU and running the CTC function...

Also, CLARIN mobile corpus seems to be too small for advanced network

In [1]:
#!/usr/bin/python3

# For demonstration purposes - Paweł Tomasik
# for CLARIN_MOBILE - generally it is unnormalized

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from keras.models import Model
from keras.layers import LSTM, Conv1D, Dropout, LeakyReLU, Dense, Input, Lambda, TimeDistributed, Flatten, Conv2D, BatchNormalization, GRU, Bidirectional
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.initializers import Orthogonal
from keras.callbacks import Callback

from keras.backend import ctc_batch_cost, expand_dims
import keras.backend as K
import tensorflow as tf

import editdistance  # For digit error rate
import keras
import librosa
import numpy as np
import os

import gc
X = Y = None
gc.collect()

LENGTH = 1700
RECS = 1384 * 4
TRANSL = 307

X = np.zeros([RECS, LENGTH, 24], np.float32)
Y = np.zeros([RECS, TRANSL], np.int16)
counter = 0

for i in range(4):
    Xpart = np.load("datasets/clarin-long/data/clarin-mfcc-rec-aligned-PCEN-MFCC-{}.npy".format(i))
    Ypart = np.load("datasets/clarin-long/data/clarin-mfcc-trans-aligned-PCEN-MFCC-{}.npy".format(i))
    recs = Xpart.shape[0]
    reclen = Xpart.shape[1]
    translen = Ypart.shape[1]
    print(Xpart.shape, Ypart.shape)
    X[counter : counter + recs, :reclen, :] = Xpart
    Y[counter : counter + recs, :translen] = Ypart
    counter += recs
    
print(counter, RECS)
counter //= 32
counter *= 32

X = X[:counter]
Y = Y[:counter]

Xpart.shape

#X = np.clip(X, -3, 3)
MEAN = X.mean()
STD = X.std()
print(MEAN, STD)

valid = np.random.random(X.shape[0]) > 0.9

/venv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


(1377, 1109, 24) (1377, 158)
(1382, 1302, 24) (1382, 183)
(1381, 1456, 24) (1381, 198)
(1383, 1603, 24) (1383, 232)
5523 5536
0.15655953 1.3286684


In [2]:
import matplotlib.pyplot as plt
first = lambda x: len(x) - np.where(x[::-1].cumsum())[0][0]
first(X[0])

-13108

In [3]:
mc = keras.callbacks.ModelCheckpoint('models/mfcc-ctc-{epoch:08d}-bigger.h5', 
                                     save_weights_only=False, period=5)

from keras.constraints import max_norm

class MyCallback(keras.callbacks.Callback):
    def __init__(self, fname):
        self.fname = fname
        super(MyCallback, self).__init__()
        
    def on_epoch_end(self, epoch, logs={}):
        content = "Epoch: {}, loss: {}\n".format(epoch, logs.get('loss'))
        with open(self.fname, "a") as f:
            f.write(content)

class StopOnConvergence(Callback):
    def __init__(self, max_repetitions=10):
        super().__init__()
        self.max_repetitions = max_repetitions

    def on_train_begin(self, logs=None):
        self.repetitions = 0
        self.last_loss = np.inf

    def on_epoch_end(self, batch, logs=None):
        logs = logs or {}
        loss = logs.get('val_loss')
        if loss is not None:
            if loss > self.last_loss:
                self.repetitions += 1
            else:
                self.last_loss = loss
                self.repetitions = 0
            if self.repetitions > self.max_repetitions:
                self.model.stop_training = True

            
logger = MyCallback("./training-bigger-log.txt")
NFEATS = 24

def mk_model(max_label_length):
    feature_input = Input(shape = (None, NFEATS))
    layer = Lambda(lambda x: (x - MEAN) / STD)(feature_input)
    layer = Lambda(K.expand_dims)(layer)
    layer_1 = Conv2D(12, [5,1], activation='linear', strides=(2,1), kernel_initializer=Orthogonal(), padding='same')(layer)
    layer_2 = LeakyReLU(0.01)(layer_1)
    layer = BatchNormalization()(layer_2)
    layer_1 = Conv2D(16, [5,1], activation='linear', strides=(2,1), kernel_initializer=Orthogonal(), padding='same')(layer)
    layer_2 = LeakyReLU(0.01)(layer_1)
    layer = BatchNormalization()(layer_2)
    layer = TimeDistributed(Flatten())(layer)
    layer_1 = Conv1D(512, 5, activation='linear', kernel_initializer=Orthogonal(), padding='same')(layer)
    layer_2 = LeakyReLU(0.01)(layer_1)
    layer = BatchNormalization()(layer_2)
    for i in range(7):
        layer = Bidirectional(GRU(256, return_sequences = True, recurrent_dropout=0.01, kernel_initializer=Orthogonal(), activation='linear'))(layer)
        layer = LeakyReLU(0.01)(layer)
        layer = BatchNormalization()(layer)
    layer = Dense(1024)(layer)
    layer = LeakyReLU(0.01)(layer)
    layer = BatchNormalization()(layer)
    layer_15 = Dense(NPHONES + 1, activation = 'softmax')(layer)
    label_input = Input(shape = (max_label_length,))
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64')
    loss_lambda = Lambda(lambda args:ctc_batch_cost(*args), output_shape=(1,), name='ctc')([label_input, layer_15, input_length, label_length])
    model = Model([feature_input, label_input, input_length, label_length], [loss_lambda])
    model.summary()
    predictive = Model(feature_input, layer_15)
    return model, predictive

def train(model, trainX, trainy, trainXl, trainyl, epochs = 50):
    # important: batch_size=1 bugs Tensorflow
    optimizer = Adam(0.0003, clipnorm=1.)
    model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=optimizer)
    return model.fit([trainX[~valid], trainy[~valid], trainXl[~valid], trainyl[~valid]], np.zeros(trainX[~valid].shape[0]), epochs = epochs,
                     batch_size = 32, callbacks=[mc, logger, StopOnConvergence(4)], validation_data=[[trainX[valid], trainy[valid], trainXl[valid], trainyl[valid]], np.zeros(trainX[valid].shape[0])])

def validate(predictions, valid_length, groundtruth, target_length):
    predictions = keras.backend.ctc_decode(predictions, valid_length, False, 1)
    predictions = predictions[0][0].eval(session=keras.backend.get_session())
    DERs = []
    for index in range(predictions.shape[0]):
        dist = float(editdistance.eval(
            [x for x in predictions[index, :] if x != -1],
            [x for x in groundtruth[index, :] if x != NPHONES]))
        DER = dist / target_length[index]
        DERs.append((DER, target_length[index]))
    return DERs

def try_else(exp, exp_else):
    try:
        return exp()
    except:
        return exp_else

if __name__=='__main__':
    data = X, Y
    NPHONES = Y.max()
    NFEATS = data[0].shape[2]
    X_lens = np.array([try_else(
            (lambda:np.where((x).mean(1) == (x).std(1))[0][0]),
            X.shape[1])
        for x in X])
    X_lens = np.ceil(X_lens / 4.0)
    Y_lens = np.array([np.where(x == NPHONES)[0] for x in data[1]])
    Y_lens = np.array([x[0] if len(x) else 0 for x in Y_lens])
    print(X.shape, Y.shape, X_lens.shape, Y_lens.shape)
    data = data[0][np.where(Y_lens)], data[1][np.where(Y_lens)], X_lens[np.where(Y_lens)].reshape(-1, 1), Y_lens[np.where(Y_lens)].reshape(-1, 1)
    print(list(map(lambda x:x.shape, data)))
    trn, predict = mk_model(data[1].shape[1])
    train(trn, *data, epochs=100) # at 300 it makes sensible predictions

(5504, 1700, 24) (5504, 307) (5504,) (5504,)
[(5504, 1700, 24), (5504, 307), (5504, 1), (5504, 1)]
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 24)     0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, None, 24)     0           input_1[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, None, 24, 1)  0           lambda_1[0][0]                   
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, 24, 12) 72          lambda_2[0][0]                   
__________

Train on 4941 samples, validate on 563 samples
Epoch 1/100
4941/4941 [==============================] - 874s 177ms/step - loss: 276.9203 - val_loss: 207.0691
Epoch 2/100
4941/4941 [==============================] - 859s 174ms/step - loss: 142.3871 - val_loss: 138.8729
Epoch 3/100
4941/4941 [==============================] - 859s 174ms/step - loss: 113.8245 - val_loss: 120.0619
Epoch 4/100
4941/4941 [==============================] - 858s 174ms/step - loss: 97.7254 - val_loss: 111.3230
Epoch 5/100
4941/4941 [==============================] - 859s 174ms/step - loss: 85.3055 - val_loss: 111.2841
Epoch 6/100
4941/4941 [==============================] - 859s 174ms/step - loss: 74.3474 - val_loss: 109.3113
Epoch 7/100
4941/4941 [==============================] - 859s 174ms/step - loss: 64.9797 - val_loss: 113.7770
Epoch 8/100
4941/4941 [==============================] - 859s 174ms/step - loss: 57.2033 - val_loss: 118.5965
Epoch 9/100
4941/4941 [==============================] - 860s 174ms/st

In [4]:
pr = predict.predict(X[:1])
pr = K.ctc_decode(pr, X_lens[:1])[0][0]
pr.eval(session=K.get_session())

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


array([[ 2, 16, 31, 32, 20, 16, 15,  2, 20, 28, 32, 23,  8, 28,  7,  4,
        28,  8, 20, 36,  6,  5, 21, 15, 10,  8, 34,  2,  8, 28,  7,  4,
        28,  2,  5, 30, 13,  4,  2, 32, 20, 20, 15, 20,  4, 34, 12, 31,
        15, 13,  7, 19, 15, 14, 25,  4,  2, 20, 16, 23,  4, 16, 32, 17,
         2]])

In [5]:
Y[:1]

array([[ 2, 31, 32, 20, 16, 15,  2, 20, 28, 32, 23,  8, 28,  7,  4,  2,
        28,  8, 20, 36,  6,  5, 21, 15, 10,  8, 34,  2,  8, 28,  2,  7,
         4, 28,  5,  2,  7, 13,  4,  2, 32,  2, 20, 20, 15, 20,  4,  2,
        34, 12, 31, 15, 13,  7,  2, 19, 15, 14, 25,  4,  2, 20, 16, 23,
         4, 16,  2, 32, 17,  2, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 

In [6]:
import editdistance
editdistance.eval(list(pr.eval(session=K.get_session())[0]), list(Y[0, :Y_lens[0]])) / Y_lens[0]

0.14285714285714285

In [16]:
pr = predict.predict(X[valid][:160])
pr = K.ctc_decode(pr, X_lens[valid][:160])[0][0]
hypos = [list(x) for x in pr.eval(session=K.get_session())]
lens = [x.index(-1) if -1 in x else len(x) for x in hypos]
hypos = [x[:lim] for x, lim in zip(hypos, lens)]
gts = [list(x) for x in Y[valid][:160]]
lens = [x.index(37) if 37 in x else len(x) for x in gts]
gts = [x[:lim] for x, lim in zip(gts, lens)]
[editdistance.eval(gt, hypo) / len(gt) for gt, hypo in zip(gts, hypos)]

[0.3023255813953488,
 0.25,
 0.29213483146067415,
 0.358974358974359,
 0.31645569620253167,
 0.2786885245901639,
 0.20238095238095238,
 0.2878787878787879,
 0.32727272727272727,
 0.3287671232876712,
 0.15384615384615385,
 0.3723404255319149,
 0.4146341463414634,
 0.24444444444444444,
 0.22727272727272727,
 0.2222222222222222,
 0.22666666666666666,
 0.3620689655172414,
 0.32954545454545453,
 0.2898550724637681,
 0.4375,
 0.3563218390804598,
 0.35353535353535354,
 0.2894736842105263,
 0.21333333333333335,
 0.2878787878787879,
 0.4519230769230769,
 0.2903225806451613,
 0.4537037037037037,
 0.421875,
 0.2463768115942029,
 0.21052631578947367,
 0.2073170731707317,
 0.45555555555555555,
 0.45161290322580644,
 0.358974358974359,
 0.21739130434782608,
 0.3064516129032258,
 0.32558139534883723,
 0.5056179775280899,
 0.4146341463414634,
 0.32222222222222224,
 0.313953488372093,
 0.12244897959183673,
 0.3511450381679389,
 0.27692307692307694,
 0.39344262295081966,
 0.37037037037037035,
 0.3118279

In [17]:
np.array(_).mean()  # on whole set: 0.1373374549857606

0.3173849783335632

In [9]:
K.zeros([2]).shape.as_list() == [2]

True

In [10]:
trn.optimizer.weights

[<tf.Variable 'Adam/iterations:0' shape=() dtype=int64_ref>,
 <tf.Variable 'training/Adam/Variable:0' shape=(5, 1, 1, 12) dtype=float32_ref>,
 <tf.Variable 'training/Adam/Variable_1:0' shape=(12,) dtype=float32_ref>,
 <tf.Variable 'training/Adam/Variable_2:0' shape=(12,) dtype=float32_ref>,
 <tf.Variable 'training/Adam/Variable_3:0' shape=(12,) dtype=float32_ref>,
 <tf.Variable 'training/Adam/Variable_4:0' shape=(5, 1, 12, 16) dtype=float32_ref>,
 <tf.Variable 'training/Adam/Variable_5:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'training/Adam/Variable_6:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'training/Adam/Variable_7:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'training/Adam/Variable_8:0' shape=(5, 384, 512) dtype=float32_ref>,
 <tf.Variable 'training/Adam/Variable_9:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'training/Adam/Variable_10:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'training/Adam/Variable_11:0' shape=(512,) dtype=float32_ref>,
 <tf.Variab

In [11]:
K.variable([9]).eval(session=K.get_session())

array([9.], dtype=float32)

In [12]:
predict.save("models/pcen_model.h5")

In [13]:
from keras.models import Model
from keras.layers import LSTM, Conv1D, Dropout, LeakyReLU, Dense, Input, Lambda, TimeDistributed, Flatten, Conv2D, BatchNormalization, GRU, Bidirectional
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.initializers import Orthogonal
from keras.callbacks import Callback

from keras.backend import ctc_batch_cost, expand_dims
import keras.backend as K


In [14]:
a = np.array([[1,2,1.1,2.1,3.3],[1.5,-2,1.12,2.12,-30]])
np.linalg.det(a.T @ a)

-2.9425586192663785e-42

In [22]:
lang = ['tS',
  'dz',
  'sil',
  'en',
  'e',
  'I',
  'v',
  'S',
  'i',
  'r',
  'dzi',
  'on',
  'f',
  'n',
  'si',
  'o',
  't',
  'k',
  'l',
  'ts',
  's',
  'x',
  'z',
  'ni',
  'b',
  'j',
  'zi',
  'g',
  'p',
  'dZ',
  'Z',
  'tsi',
  'a',
  'd',
  'm',
  'w',
  'u']

In [26]:
pr = predict.predict(X[valid][:1])
pr = K.ctc_decode(pr, X_lens[valid][:1])[0][0]
letters = pr.eval(session=K.get_session())
[lang[x] for x in letters[0]]

['sil',
 'l',
 'i',
 'sil',
 'g',
 'j',
 'a',
 'k',
 'i',
 's',
 's',
 'p',
 'o',
 's',
 'u',
 'k',
 'p',
 'S',
 'I',
 'tS',
 'I',
 'ni',
 'i',
 'm',
 'e',
 'm',
 'si',
 'e',
 'd',
 'o',
 'r',
 'e',
 'f',
 'o',
 'r',
 'n',
 'I',
 'x',
 'k',
 'o',
 'n',
 's',
 't',
 'I',
 't',
 'u',
 'ts',
 'i',
 'n',
 'e',
 'j',
 'sil',
 'n',
 'a',
 'm',
 'j',
 'a',
 'r',
 'j',
 'm',
 'v',
 'r',
 'sil',
 'a',
 's',
 'ni',
 'i',
 'x',
 'm',
 'o',
 'Z',
 'l',
 'i',
 'v',
 'w',
 'o',
 'si',
 'tsi',
 'i',
 'sil']

In [25]:
[lang[x] for x in Y[valid][0][:Y_lens[valid][0]]]

['sil',
 'v',
 'sil',
 'j',
 'a',
 'k',
 'i',
 'si',
 's',
 'p',
 'o',
 's',
 'u',
 'p',
 'sil',
 'p',
 'S',
 'I',
 'tS',
 'I',
 'ni',
 'i',
 'w',
 'sil',
 'e',
 'm',
 'si',
 'e',
 'sil',
 'd',
 'o',
 'r',
 'e',
 'f',
 'sil',
 'o',
 'r',
 'm',
 'I',
 'sil',
 'k',
 'o',
 'n',
 's',
 't',
 'I',
 'sil',
 't',
 'u',
 'ts',
 'I',
 'j',
 'n',
 'e',
 'j',
 'sil',
 'n',
 'a',
 'm',
 'j',
 'sil',
 'a',
 'r',
 'en',
 'sil',
 'v',
 'w',
 'a',
 's',
 'n',
 'I',
 'x',
 'm',
 'o',
 'sil',
 'Z',
 'l',
 'i',
 'sil',
 'v',
 'o',
 'sil',
 'si',
 'tsi',
 'i',
 'sil']